In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re 
import  requests
from selenium.webdriver.common.by import By 
from tqdm import tqdm  
import pandas as pd 
import pickle 

In [ ]:
def find_soup(url):
    response=requests.get(url)
    content=response.content
    soup = BeautifulSoup(content, 'html.parser')  
    name=soup.find('h1') 
    tele=soup.find_all('tr')  
    desc=soup.find_all('div' , class_='object_desc')  
    tele=list(map(str,tele))  
    name=str(name)
    desc=str(desc[0])
    return name,tele ,desc 

In [ ]:
def clean_email (x) : 
    t=re.search('">(.*)</a>' ,x).group(1) 
    return t 

In [ ]:
def clean_adress (x): 
    t=re.search('<tr> <td>ADRES:</td> <td>(.*)</td> </tr>' ,x).group(1)  #<tr> <td>ADRES:</td> <td>
    return t  

def clean_telephone (x): 
    t=re.search('">(.*)</a> </div> </td> </tr>' ,x).group(1)  
    return t  


def clean_name (x): 
    t=re.search('<h1> (.*)</h1>' ,x).group(1)  
    return t 


def clean_name2 (x): 
    t=re.search('<h1>(.*)</h1>' ,x).group(1)  
    return t 


def clean_email (x) : 
    t=re.search('">(.*)</a>' ,x).group(1) 
    return t 


def clean_courts (x):  #<div class="object_desc"> <p><strong>Obiekt:</strong></p><p>/</p> </div>
    try: 
        t=re.search('<div class="object_desc"> <p><strong>Obiekt:</strong></p><p>(.*)</p><p><strong>' ,x).group(1)  
    except: 
        try : 
            t=re.search('<div class="object_desc"> <p><strong>Obiekt:</strong></p> <p>(.*)</p> <p><strong>' ,x).group(1) 
        except: 
            try :    
                t=re.search('<div class="object_desc"> <p><strong>Obiekt:</strong></p><p>(.*)</p> </div>' ,x).group(1) 
            except: 
                try:
                    t=re.search('ong>Obiekt:</strong></p><p>(.*)</p><p><strong>' ,x).group(1) 
                except: 
                    try:
                        t=re.search('ong>Obiekt:</strong></p><p>(.*)</p>' ,x).group(1)  
                    except:  
                        try:
                            t=re.search('ong>Obiekt:</strong></p><p>(.*)\n ' ,x).group(1) 
                        except: 
                            t=re.search('rong>Obiekt:</strong></p><p>(.*)\n' ,x).group(1) 
                        
    return t 


def clean_activity(x): 
    if 'Zajęcia' in x : 
        try:
            t=re.search('Zajęcia m.in.:</strong></p><p>(.*)</p> </div>' ,x).group(1) #Zajęcia m.in.:</strong></p> <p>  
        except: 
            try:
                t=re.search('Zajęcia m.in.:</strong></p> <p>(.*)</p> </div>' ,x).group(1)#</p> </div> 
            except: 
                try: 
                    t=re.search('in.:</strong></p><p>(.*)\n' ,x).group(1)  
                except:  
                    try:
                        t=re.search('<strong>Zajęcia m.in\n(.*)</strong></p></div>' ,x).group(1) 
                    except: 
                        t=re.search('>(.*)</div>',description).group(1) 
    else : 
        t=None 
    return t  

def clean_web(x):
    t=re.search('<tr> <td>WWW:</td> <td> <p><a href="(.*)" target="_blank">' ,x).group(1)  
    return t  

#<div class="object_desc"> <p><strong>Obiekt:</strong></p><p>kort tenisowy, sala sportowa, siłownia wewnętrzna, orlik,
#boisko do piłki nożnej, basen kryty</p><p><strong>Zajęcia m.in.:</strong></p><p>tenis ziemny,
#sztuki walki, piłka nożna, koszykówka, siatkówka, piłka ręczna, futsa\n    </p></div>'



In [ ]:
def  clean_all(description,p):  
    if len(p)==2 : 
        try: 
            court=clean_courts(description) 
            tel=clean_telephone(p[1]) 
            adresse=clean_adress(p[0]) 
            activ=clean_activity(description)
            web=None  
            email=None 
        except: 
            court=clean_courts(description) 
            tel=None 
            adresse=clean_adress(p[0]) 
            activ=clean_activity(description)
            web=clean_web(p[1])  
            email=None 
    if len(p)==1 : 
        court=clean_courts(description) 
        adresse=clean_adress(p[0]) 
        tel=None 
        web=None  
        activ=None 
        email=None 
    if len(p)==3 : 
        try: 
            adresse=clean_adress(p[0]) 
            tel=clean_telephone(p[2]) 
            web=clean_web(p[1])
            activ=clean_activity(description)
            court=clean_courts(description) 
            email=None 
        except: 
            try: 
                adresse=clean_adress(p[0]) 
                tel=clean_telephone(p[2]) 
                web=None 
                activ=clean_activity(description)
                court=clean_courts(description) 
                email=clean_email(p[1]) 
            except: 
                adresse=clean_adress(p[0]) 
                tel=None
                web=clean_web(p[2]) 
                activ=clean_activity(description)
                court=clean_courts(description) 
                email=clean_email(p[1]) 
    if len(p)==4 : 
        adresse=clean_adress(p[0])
        tel=clean_telephone(p[3]) 
        web=clean_web(p[2]) 
        email=clean_email(p[1]) 
        activ=clean_activity(description)
        court=clean_courts(description) 
    return adresse ,tel,email ,web,court,activ 


#<div class="object_desc"> <p><strong>Obiekt:</strong></p><p>kort tenisowy, sala sportowa, siłownia wewnętrzna, orlik,
#boisko do piłki nożnej, basen kryty</p><p><strong>Zajęcia m.in.:</strong></p><p>tenis ziemny,
#sztuki walki, piłka nożna, koszykówka, siatkówka, piłka ręczna, futsa\n    </p></div>'

In [ ]:
links=[]
'''
non_general_links=[]
name_col=[] 
tel_col=[] 
adress_col=[] 
courts_col=[]
activities_col=[] 
web_col=[] 
n=0 
'''  
gab=[] 
for page in tqdm(range(92)): 
    time.sleep(1)
    url="https://www.krakow.pl/instcbi/247778,,2589,"+str(page)+",wyszukiwarka.html"
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options, executable_path="/Users/midou/Desktop/web_scraping/chromedriver")
    driver.get(url)
    elems = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".article-map-points__table__content [href]"))) 
    links = [elem.get_attribute('href') for elem in elems]   
    bid=links[::2] 
    gab.append(bid)  
    driver.close() 
    
    '''
    for link in bid:
        try: 
            name,telephone,description=find_soup(link)  
            
            try : 
                name=clean_name(name) 
            "https://www.krakow.pl/instcbi/247778,,2589,0,wyszukiwarka.html"
            except: 
                name=clean_name2(name) 
          
            if len(p)==2 : 
                court=clean_courts(description) 
                tel=clean_telephone(p[1]) 
                adresse=clean_adress(p[0]) 
                activ=clean_activity(description)
                web=None 
            elif len(p)==1 : 
                court=clean_courts(description) 
                adresse=clean_adress(p[0]) 
                tel=None 
                web=None  
                activ=None
            else : 
                adresse=clean_adress(p[0]) 
                tel=clean_telephone(p[2]) 
                web=clean_web(p[1])
                activ=clean_activity(description)
        
        
            if n >0 : 
                if name != privous_name : 
                    name_col.append(name)  
                    tel_col.append(tel)
                    adress_col.append(adresse)  
                    courts_col.append(court) 
                    activities_col.append(activ)  
                    web_col.append(web)
                    privous_name=name 
                    n+=1 
                    time.sleep(0.3) 
                else :  
                    pass  
            else : 
                name_col.append(name)  
                tel_col.append(tel)
                adress_col.append(adresse)  
                courts_col.append(court) 
                activities_col.append(activ)  
                web_col.append(web) 
                n+=1 
                privous_name=name
                time.sleep(0.3)
        except:
            print("non_genral_link") 
            n+=1 
            non_general_links.append(link)
            ''' 
    
        
        
        
        
        
        
        
        
        
    
    
    

  0%|          | 0/92 [00:00<?, ?it/s]/var/folders/4k/2v2nn1sn1k1f7dtmr6ny96vh0000gn/T/ipykernel_10174/355656777.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path="/Users/midou/Desktop/web_scraping/chromedriver")
100%|██████████| 92/92 [11:48<00:00,  7.70s/it]


In [ ]:
flat_list = []
for sublist in gab:
    for item in sublist:
        flat_list.append(item) 

In [ ]:
non_general_links=[]
name_col=[] 
tel_col=[] 
adress_col=[] 
courts_col=[]
activities_col=[] 
web_col=[] 
email_col=[]
n=0 
for link in tqdm(flat_list):
    try:
        name,p,description=find_soup(link)  
            
        try : 
            name=clean_name(name) 
            
        except: 
            name=clean_name2(name) 
          
        adresse ,tel,email ,web,court,activ=clean_all(description,p)    
        if n >0 : 
            if name != privous_name : 
                name_col.append(name)  
                tel_col.append(tel)
                adress_col.append(adresse)  
                courts_col.append(court) 
                activities_col.append(activ)  
                web_col.append(web) 
                email_col.append(email)
                privous_name=name 
                n+=1 
                time.sleep(0.3) 
            else :  
                pass  
        else : 
            name_col.append(name)  
            tel_col.append(tel)
            adress_col.append(adresse)  
            courts_col.append(court) 
            activities_col.append(activ)  
            web_col.append(web)  
            email_col.append(email)
            n+=1 
            privous_name=name
            time.sleep(1.3)
    except: 
        non_general_links.append(link) 
        n+=1

100%|██████████| 1832/1832 [26:02<00:00,  1.17it/s] 


In [ ]:
non_general_links

['https://www.krakow.pl/instcbi/247778,inst,77082,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,74820,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,76454,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,75004,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,76301,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,76489,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,,2589,6,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,6,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,6,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,6,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,6,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,6,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,inst,74979,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,76274,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/

In [ ]:
List_frame=[name_col,tel_col,web_col,adress_col,email_col ,courts_col,activities_col]

In [ ]:
List_frame=[name_col,tel_col,web_col,adress_col,email_col ,courts_col,activities_col]
df=pd.DataFrame(List_frame, index=['Facility.Name', 'Facility.Tel_Number','Facility.Web_site',
                                    'Facility.Adress','Facility.email','Facility.Courts','Facility.Sport_activities']).T

In [ ]:
df

,Facility.Name,Facility.Tel_Number,Facility.Web_site,Facility.Adress,Facility.email,Facility.Courts,Facility.Sport_activities
0,Klub Sportowy Korona,+48 12 656 53 89,None,"ul. Kalwaryjska 9, 30-504 Kraków",None,"ścianka wspinaczkowa, basen kryty, sala gimnas...","wspinaczka, pływanie, gimnastyka, koszykówka, ..."
1,Ludowy Klub Sportowy Tonianka,+48 604 510 041,"http://tonianka.futbolowo.pl/menu,2,o-klubie.html","ul. Skotnica 1, 30-394 Kraków",None,"boisko do piłki nożnej, boisko wielofunkcyjne","piłka nożna, koszykówka"
2,A3 Sport Fitness,+48 12 416 10 19,None,"ul. Konecznego 4, 31-216 Kraków",None,siłownia wewnętrzna,fitness
3,Active Niuton Ośrodek Promocji Zdrowia,+48 500 210 387,http://activeniuton.pl,"Osiedle Teatralne 10, 31-943 Kraków",None,siłownia wewnętrzna,fitness
4,Akademia Górniczo Hutnicza,+48 12 617 48 40,http://www.basen.agh.edu.pl/?page_id=187,"ul. Buszka 4, 30-150 Kraków",None,"centrum zabaw, sala sportowa, basen kryty, krę...","plac zabaw dla dzieci, Tai Chi, fitness, pływa..."
...,...,...,...,...,...,...,...
604,ZS Przemysłu Spożywczego,+48 12 422 38 66,http://zsps.krakow.pl/,"ul. Miechowity 6, 31-475 Kraków",None,sala gimnastyczna,zajęcia sportowe
605,"ZSOMS im. Mikołaja Kopernika (XXXVIII LOMS, SP...",+48 12 411 29 48,None,"ul. Grochowska 20, 31-521 Kraków",None,"sala gimnastyczna, hala sportowa, sala fitness...","unihokej, zajęcia sportowe, fitness, gimnastyk..."
606,ZSS nr 3,+48 12 333 94 99,http://www.zssnr3krakow.pl/,"ul. Wielicka 265, 30-663 Kraków",None,sala gimnastyczna,zajęcia sportowe
607,ZSZ Polskiego Górnictwa Naftowego i Gazownictw...,+48 12 422 49 55,https://brzozowa5.edu.pl/,"ul. Brzozowa 5, 31-050 Kraków",None,sala gimnastyczna,zajęcia sportowe


In [ ]:
vertical_concat.to_csv('Krkaw_all_facilities3.csv') 

In [ ]:
import pickle

In [ ]:
with open('parro__1t.pkl', 'rb') as f:
    flat_list = pickle.load(f) 
    
    
    

In [ ]:
non_general_links2=[]
name_col=[] 
tel_col=[] 
adress_col=[] 
courts_col=[]
activities_col=[] 
web_col=[] 
email_col=[]
n=0 
for link in tqdm(non_general_links):
    try:
        name,p,description=find_soup(link)  
            
        try : 
            name=clean_name(name) 
            
        except: 
            name=clean_name2(name) 
          
        adresse ,tel,email ,web,court,activ=clean_all(description,p)    
        if n >0 : 
            if name != privous_name : 
                name_col.append(name)  
                tel_col.append(tel)
                adress_col.append(adresse)  
                courts_col.append(court) 
                activities_col.append(activ)  
                web_col.append(web) 
                email_col.append(email)
                privous_name=name 
                n+=1 
                time.sleep(0.3) 
            else :  
                pass  
        else : 
            name_col.append(name)  
            tel_col.append(tel)
            adress_col.append(adresse)  
            courts_col.append(court) 
            activities_col.append(activ)  
            web_col.append(web)  
            email_col.append(email)
            n+=1 
            privous_name=name
            time.sleep(1.3)
    except: 
        non_general_links2.append(link) 
        n+=1

100%|██████████| 217/217 [05:02<00:00,  1.40s/it]


In [ ]:
non_general_links2

119

In [ ]:
List_frame=[name_col,tel_col,web_col,adress_col,email_col ,courts_col,activities_col]
df2=pd.DataFrame(List_frame, index=['Facility.Name', 'Facility.Tel_Number','Facility.Web_site',
                                    'Facility.Adress','Facility.email','Facility.Courts','Facility.Sport_activities']).T

In [ ]:
non_general_links2

['https://www.krakow.pl/instcbi/247778,inst,77082,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,74820,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,,2589,6,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,6,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,6,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,6,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,6,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,6,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,14,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,14,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,14,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,14,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,,2589,14,wyszukiwarka.html#',
 'https://www.krakow.pl/instcbi/247778,inst,76441,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/24777

In [ ]:
good_url=[] 
bad_url=[]
for url in tqdm(non_general_links2) : 
    try : 
        name,p,description=find_soup(url)    
        adresse ,tel,email ,web,court,activ=clean_all(description,p)   
        good_url.append(url) 
    except: 
        bad_url.append(url) 
        

100%|██████████| 119/119 [03:38<00:00,  1.83s/it]


In [ ]:
for url in tqdm(good_url) :  
        name,p,description=find_soup(url)    
        adresse ,tel,email ,web,court,activ=clean_all(description,p)  
        try : 
            name=clean_name(name) 
        except: 
            name=clean_name2(name)
        
        

100%|██████████| 16/16 [00:09<00:00,  1.61it/s]


In [ ]:
to_scrap=[] 
for i in bad_ur

In [ ]:
List_frame 
urls = []
for sublist in List_frame:
    for item in sublist: 
        urls.append(item)

In [ ]:
len(bad_url2) 

41

In [ ]:
non_general_links3=[]
name_col=[] 
tel_col=[] 
adress_col=[] 
courts_col=[]
activities_col=[] 
web_col=[] 
email_col=[]
n=0 
for link in tqdm(good_url):
    
        name,p,description=find_soup(link)  
            
        try : 
            name=clean_name(name) 
            
        except: 
            name=clean_name2(name) 
          
        adresse ,tel,email ,web,court,activ=clean_all(description,p)    
        if n >0 : 
            if name != privous_name : 
                name_col.append(name)  
                tel_col.append(tel)
                adress_col.append(adresse)  
                courts_col.append(court) 
                activities_col.append(activ)  
                web_col.append(web) 
                email_col.append(email)
                privous_name=name 
                n+=1 
                time.sleep(0.3) 
            else :  
                pass  
        else : 
            name_col.append(name)  
            tel_col.append(tel)
            adress_col.append(adresse)  
            courts_col.append(court) 
            activities_col.append(activ)  
            web_col.append(web)  
            email_col.append(email)
            n+=1 
            privous_name=name
            time.sleep(1.3) 
'''        
    except: 
        non_general_links3.append(link) 
        n+=1 
        print('problem') 
    '''

100%|██████████| 16/16 [00:11<00:00,  1.40it/s]


"        \n    except: \n        non_general_links3.append(link) \n        n+=1 \n        print('problem') \n    "

In [ ]:
List_frame=[name_col,tel_col,web_col,adress_col,email_col ,courts_col,activities_col]
df3=pd.DataFrame(List_frame, index=['Facility.Name', 'Facility.Tel_Number','Facility.Web_site',
                                    'Facility.Adress','Facility.email','Facility.Courts','Facility.Sport_activities']).T

In [ ]:
vertical_concat = pd.concat([df, df2,df3], axis=0)

In [ ]:
vertical_concat

,Facility.Name,Facility.Tel_Number,Facility.Web_site,Facility.Adress,Facility.email,Facility.Courts,Facility.Sport_activities
0,Klub Sportowy Korona,+48 12 656 53 89,None,"ul. Kalwaryjska 9, 30-504 Kraków",None,"ścianka wspinaczkowa, basen kryty, sala gimnas...","wspinaczka, pływanie, gimnastyka, koszykówka, ..."
1,Ludowy Klub Sportowy Tonianka,+48 604 510 041,"http://tonianka.futbolowo.pl/menu,2,o-klubie.html","ul. Skotnica 1, 30-394 Kraków",None,"boisko do piłki nożnej, boisko wielofunkcyjne","piłka nożna, koszykówka"
2,A3 Sport Fitness,+48 12 416 10 19,None,"ul. Konecznego 4, 31-216 Kraków",None,siłownia wewnętrzna,fitness
3,Active Niuton Ośrodek Promocji Zdrowia,+48 500 210 387,http://activeniuton.pl,"Osiedle Teatralne 10, 31-943 Kraków",None,siłownia wewnętrzna,fitness
4,Akademia Górniczo Hutnicza,+48 12 617 48 40,http://www.basen.agh.edu.pl/?page_id=187,"ul. Buszka 4, 30-150 Kraków",None,"centrum zabaw, sala sportowa, basen kryty, krę...","plac zabaw dla dzieci, Tai Chi, fitness, pływa..."
...,...,...,...,...,...,...,...
34,Wojskowy Klub Sportowy Wawel,+48 12 637 27 91,http://wkswawel.pl,"ul. Podchorążych 3, 30-084 Kraków",None,"stadion lekkoatletyczny, strzelnica, kort teni...","lekkoatletyka, strzelectwo sportowe, tenis zie..."
35,"Zespół Szkolno-Przedszkolny nr 1 (SP nr 47, P ...",+48 12 653 21 45,https://sp47krakow.edu.pl/,"ul. Myśliwska 64, 30-718 Kraków",None,"boisko wielofunkcyjne, zeskok do skoku w dal, ...","koszykówka, lekkoatletyka, biegi, lekkoatletyk..."
0,ogródek jordanowski,+48 12 201 02 40,https://zzm.krakow.pl/index.php,"ul. Kantorowicka, 31-763 Kraków",None,"stół do tenisa stołowego, plac zabaw dla dziec...",None
1,Orlik 2012 – kompleks boisk sportowych,+48 12 411 15 28,None,"ul. Sadzawki 1, 31-465 Kraków",sp64krakow.pl,"orlik, boisko wielofunkcyjne","piłka nożna, koszykówka, siatkówka"


In [ ]:
non_general_links_33=[]
name_col=[] 
tel_col=[] 
adress_col=[] 
courts_col=[]
activities_col=[] 
web_col=[] 
email_col=[]
n=0 
for link in tqdm(bad_url2):
    try:
        name,p,description=find_soup(link)  
            
        try : 
            name=clean_name(name) 
            
        except: 
            name=clean_name2(name) 
          
        adresse ,tel,email ,web,court,activ=clean_all(description,p)    
        if n >0 : 
            if name != privous_name : 
                name_col.append(name)  
                tel_col.append(tel)
                adress_col.append(adresse)  
                courts_col.append(court) 
                activities_col.append(activ)  
                web_col.append(web) 
                email_col.append(email)
                privous_name=name 
                n+=1 
                time.sleep(0.3) 
            else :  
                pass  
        else : 
            name_col.append(name)  
            tel_col.append(tel)
            adress_col.append(adresse)  
            courts_col.append(court) 
            activities_col.append(activ)  
            web_col.append(web)  
            email_col.append(email)
            n+=1 
            privous_name=name
            time.sleep(1.3)
    except: 
        non_general_links_33.append(link) 
        n+=1

100%|██████████| 41/41 [01:37<00:00,  2.38s/it]


In [ ]:
len(non_general_links_33) 

8

In [ ]:
List_frame=[name_col,tel_col,web_col,adress_col,email_col ,courts_col,activities_col]
df33=pd.DataFrame(List_frame, index=['Facility.Name', 'Facility.Tel_Number','Facility.Web_site',
                                    'Facility.Adress','Facility.email','Facility.Courts','Facility.Sport_activities']).T

In [ ]:
df33

,Facility.Name,Facility.Tel_Number,Facility.Web_site,Facility.Adress,Facility.email,Facility.Courts,Facility.Sport_activities
0,AVATAR Centrala Ruchu/AcroPark/FunClimb,+48 733 434 332,https://myavatar.pl/,"ul. Sikorki 21A, 31-589 Kraków",None,"sala sportowa, siłownia wewnętrzna,","ABT, joga, kettlebell, pilates, szarfy, fitnes..."
1,Klub Sportowy Tyniec,None,http://www.kstyniec.pl/,"ul. Juranda Ze Spychowa 20, 30-398 Kraków",None,boisko do piłki nożnej,piłka nożna
2,Kryty pool Kraków,None,"http://www.deskorolki.net.pl/skateparki/2717,k...","konopnicka 28, 30-307 Kraków",None,skatepark,"jazda na BMX-ie, jazda na rolkach, jazda na de..."
3,Ludowy Klub Sportowy Zjednoczeni ZŁOMEX Branice,None,http://zlomex-branice.futbolowo.pl,"ul. Karaszewicza-Tokarzewskiego 29, 31-985 Kraków",None,boisko do piłki nożnej,piłka nożna
4,ogródek jordanowski,+48 12 201 02 40,https://zzm.krakow.pl/index.php,"ul. Dąbrowskiej, 31-800 Kraków",None,"stół do tenisa stołowego, boisko do koszykówki...",</p>
5,Siłownia Retro GYM,None,https://retrogym.business.site/,"ul. Smoleńsk 16/1, 31-112 Kraków",None,siłownia wewnętrzna,fitness
6,Skatepark Kraków ul. Wysłouchów,None,"http://www.deskorolki.net.pl/skateparki/2688,s...","ul. Wysłouchów, 30-611 Kraków",None,skatepark,"jazda na BMX-ie, jazda na rolkach, jazda na de..."
7,Skatepark Mistrzejowice,None,"http://www.freestyle.pl/news/id,42450/Skatepar...","ul. Wawelska, 31-619 Kraków",None,skatepark,"jazda na BMX-ie, jazda na rolkach, jazda na de..."
8,Skatepark os. Widok,None,https://www.atrakcjedzieciece.pl/dashboard/att...,"ul. Zarzecze 110, 30-134 Kraków",None,skatepark,"jazda na BMX-ie, jazda na rolkach, jazda na de..."
9,SP nr 138 im. Polskich Wojsk Lotniczych,None,http://sp138.pl,"ul. Kazimierza Wielkiego 3, 30-198 Kraków",sp138@interia.pl,boisko do piłki nożnej,piłka nożna


In [ ]:
non_general_links_33

['https://www.krakow.pl/instcbi/247778,inst,74820,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,76314,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,77170,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,77728,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,74832,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,81277,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,77780,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,76411,2589,instcbi.html']

In [ ]:
vertical_concat = pd.concat([df, df2,df3,df33], axis=0)

In [ ]:
vertical_concat.to_csv('krakaw_ALL_instalations.csv') 

In [ ]:
pickle.dump( non_general_links_33, open( "save_to_do_fcili.p", "wb" ) )

In [ ]:
file = open("save_to_do_fcili.p",'rb')
object_file = pickle.load(file)

In [ ]:
object_file  

['https://www.krakow.pl/instcbi/247778,inst,74820,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,76314,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,77170,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,77728,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,74832,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,81277,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,77780,2589,instcbi.html',
 'https://www.krakow.pl/instcbi/247778,inst,76411,2589,instcbi.html']

In [ ]:
name_col=['Bielany-Celiny','Krakowski park linowy','Park Planty im. F. Nowackiego',
          'SP nr 32 im. Karola Chodkiewicza','Ścieżka edukacyjna','Tor rowerowy typu pumptrack (modułowy) - Złocień'] 
tel_col=[None,'+48 514 256 647',None,None,'+48 12 201 02 40','+48 12 201 02 40' ] 
adress_col=['ul. Celiny, Kraków','ul. Widłakowa 13, 30-380 Kraków','Park Planty im. F. Nowackiego, Kraków',
              'SP nr 32 im. Karola Chodkiewicza','Mistrzejowice, Kraków','ul. Lubostroń 14, 30-383 Kraków' ] 
courts_col=['tenis stołowy, boisko i/lub bieżnia','park linowy','siłownia zewnętrzna','boisko wielofunkcyjne',
           'plac zabaw dla dzieci, ścieżka edukacyjna','obiekt rowerowy']
activities_col=[None,'park linowy',None,'zajęcia sportowe','plac zabaw dla dzieci, ścieżka edukacyjna','kolarstwo'] 
web_col=[None,None,None,None,None,'zzm.krakow.pl'] 
email_col=[None,None,None,None,None,None]

In [ ]:
List_frame=[name_col,tel_col,web_col,adress_col,email_col ,courts_col,activities_col]
df223=pd.DataFrame(List_frame, index=['Facility.Name', 'Facility.Tel_Number','Facility.Web_site',
                                    'Facility.Adress','Facility.email','Facility.Courts','Facility.Sport_activities']).T

In [ ]:
df223

,Facility.Name,Facility.Tel_Number,Facility.Web_site,Facility.Adress,Facility.email,Facility.Courts,Facility.Sport_activities
0,Bielany-Celiny,None,None,"ul. Celiny, Kraków",None,"tenis stołowy, boisko i/lub bieżnia",None
1,Krakowski park linowy,+48 514 256 647,None,"ul. Widłakowa 13, 30-380 Kraków",None,park linowy,park linowy
2,Park Planty im. F. Nowackiego,None,None,"Park Planty im. F. Nowackiego, Kraków",None,siłownia zewnętrzna,None
3,SP nr 32 im. Karola Chodkiewicza,None,None,SP nr 32 im. Karola Chodkiewicza,None,boisko wielofunkcyjne,zajęcia sportowe
4,Ścieżka edukacyjna,+48 12 201 02 40,None,"Mistrzejowice, Kraków",None,"plac zabaw dla dzieci, ścieżka edukacyjna","plac zabaw dla dzieci, ścieżka edukacyjna"
5,Tor rowerowy typu pumptrack (modułowy) - Złocień,+48 12 201 02 40,zzm.krakow.pl,"ul. Lubostroń 14, 30-383 Kraków",None,obiekt rowerowy,kolarstwo


In [ ]:
data_=pd.read_csv('krakaw_ALL_instalations.csv') 

In [ ]:
data_.columns

Index(['Unnamed: 0', 'Facility.Name', 'Facility.Tel_Number',
       'Facility.Web_site', 'Facility.Adress', 'Facility.email',
       'Facility.Courts', 'Facility.Sport_activities'],
      dtype='object')

In [ ]:
del data_['Unnamed: 0'] 

In [ ]:
data_ 

,Facility.Name,Facility.Tel_Number,Facility.Web_site,Facility.Adress,Facility.email,Facility.Courts,Facility.Sport_activities
0,Klub Sportowy Korona,+48 12 656 53 89,NaN,"ul. Kalwaryjska 9, 30-504 Kraków",NaN,"ścianka wspinaczkowa, basen kryty, sala gimnas...","wspinaczka, pływanie, gimnastyka, koszykówka, ..."
1,Ludowy Klub Sportowy Tonianka,+48 604 510 041,"http://tonianka.futbolowo.pl/menu,2,o-klubie.html","ul. Skotnica 1, 30-394 Kraków",NaN,"boisko do piłki nożnej, boisko wielofunkcyjne","piłka nożna, koszykówka"
2,A3 Sport Fitness,+48 12 416 10 19,NaN,"ul. Konecznego 4, 31-216 Kraków",NaN,siłownia wewnętrzna,fitness
3,Active Niuton Ośrodek Promocji Zdrowia,+48 500 210 387,http://activeniuton.pl,"Osiedle Teatralne 10, 31-943 Kraków",NaN,siłownia wewnętrzna,fitness
4,Akademia Górniczo Hutnicza,+48 12 617 48 40,http://www.basen.agh.edu.pl/?page_id=187,"ul. Buszka 4, 30-150 Kraków",NaN,"centrum zabaw, sala sportowa, basen kryty, krę...","plac zabaw dla dzieci, Tai Chi, fitness, pływa..."
...,...,...,...,...,...,...,...
658,SP nr 50 im. Włodzimierza Tetmajera,NaN,https://sp50krakow.edupage.org,"ul. Katowicka 28, 31-351 Kraków",sp50krakow@gmail.com,"boisko wielofunkcyjne, boisko do piłki nożnej,...","zajęcia sportowe, piłka nożna, biegi"
659,SP z Oddziałami Sportowymi nr 5 im. Polskich O...,+48 12 636 70 93,http://sp5.krakow.pl,"Aleja Kijowska 8, 30-079 Kraków",poczta@sp5.krakow.pl,"basen kryty, sala gimnastyczna, sala sportowa,...","pływanie, gimnastyka sportowa, taniec, judo, s..."
660,Studio 5,NaN,http://studio5x.pl,"ul. Gromady Grudziąż 5, 30-658 Kraków",NaN,siłownia wewnętrzna,fitness
661,XXV Liceum Ogólnokształcące im. Stefana Żeroms...,NaN,https://www.xxvlo.pl,"ul. Telimeny 9, 30-838 Kraków",sekretariat@xxvlo.pl,"sala gimnastyczna, boisko do siatkówki, boisko...","zajęcia sportowe, siatkówka, koszykówka, piłka..."


In [ ]:
vertical_concat = pd.concat([data_,df223], axis=0) 

In [ ]:
vertical_concat 

,Facility.Name,Facility.Tel_Number,Facility.Web_site,Facility.Adress,Facility.email,Facility.Courts,Facility.Sport_activities
0,Klub Sportowy Korona,+48 12 656 53 89,NaN,"ul. Kalwaryjska 9, 30-504 Kraków",NaN,"ścianka wspinaczkowa, basen kryty, sala gimnas...","wspinaczka, pływanie, gimnastyka, koszykówka, ..."
1,Ludowy Klub Sportowy Tonianka,+48 604 510 041,"http://tonianka.futbolowo.pl/menu,2,o-klubie.html","ul. Skotnica 1, 30-394 Kraków",NaN,"boisko do piłki nożnej, boisko wielofunkcyjne","piłka nożna, koszykówka"
2,A3 Sport Fitness,+48 12 416 10 19,NaN,"ul. Konecznego 4, 31-216 Kraków",NaN,siłownia wewnętrzna,fitness
3,Active Niuton Ośrodek Promocji Zdrowia,+48 500 210 387,http://activeniuton.pl,"Osiedle Teatralne 10, 31-943 Kraków",NaN,siłownia wewnętrzna,fitness
4,Akademia Górniczo Hutnicza,+48 12 617 48 40,http://www.basen.agh.edu.pl/?page_id=187,"ul. Buszka 4, 30-150 Kraków",NaN,"centrum zabaw, sala sportowa, basen kryty, krę...","plac zabaw dla dzieci, Tai Chi, fitness, pływa..."
...,...,...,...,...,...,...,...
1,Krakowski park linowy,+48 514 256 647,None,"ul. Widłakowa 13, 30-380 Kraków",None,park linowy,park linowy
2,Park Planty im. F. Nowackiego,None,None,"Park Planty im. F. Nowackiego, Kraków",None,siłownia zewnętrzna,None
3,SP nr 32 im. Karola Chodkiewicza,None,None,SP nr 32 im. Karola Chodkiewicza,None,boisko wielofunkcyjne,zajęcia sportowe
4,Ścieżka edukacyjna,+48 12 201 02 40,None,"Mistrzejowice, Kraków",None,"plac zabaw dla dzieci, ścieżka edukacyjna","plac zabaw dla dzieci, ścieżka edukacyjna"


In [ ]:
vertical_concat.set_index(vertical_concat['Facility.Name'])  

,Facility.Name,Facility.Tel_Number,Facility.Web_site,Facility.Adress,Facility.email,Facility.Courts,Facility.Sport_activities
Facility.Name,,,,,,,
Klub Sportowy Korona,Klub Sportowy Korona,+48 12 656 53 89,NaN,"ul. Kalwaryjska 9, 30-504 Kraków",NaN,"ścianka wspinaczkowa, basen kryty, sala gimnas...","wspinaczka, pływanie, gimnastyka, koszykówka, ..."
Ludowy Klub Sportowy Tonianka,Ludowy Klub Sportowy Tonianka,+48 604 510 041,"http://tonianka.futbolowo.pl/menu,2,o-klubie.html","ul. Skotnica 1, 30-394 Kraków",NaN,"boisko do piłki nożnej, boisko wielofunkcyjne","piłka nożna, koszykówka"
A3 Sport Fitness,A3 Sport Fitness,+48 12 416 10 19,NaN,"ul. Konecznego 4, 31-216 Kraków",NaN,siłownia wewnętrzna,fitness
Active Niuton Ośrodek Promocji Zdrowia,Active Niuton Ośrodek Promocji Zdrowia,+48 500 210 387,http://activeniuton.pl,"Osiedle Teatralne 10, 31-943 Kraków",NaN,siłownia wewnętrzna,fitness
Akademia Górniczo Hutnicza,Akademia Górniczo Hutnicza,+48 12 617 48 40,http://www.basen.agh.edu.pl/?page_id=187,"ul. Buszka 4, 30-150 Kraków",NaN,"centrum zabaw, sala sportowa, basen kryty, krę...","plac zabaw dla dzieci, Tai Chi, fitness, pływa..."
...,...,...,...,...,...,...,...
Krakowski park linowy,Krakowski park linowy,+48 514 256 647,None,"ul. Widłakowa 13, 30-380 Kraków",None,park linowy,park linowy
Park Planty im. F. Nowackiego,Park Planty im. F. Nowackiego,None,None,"Park Planty im. F. Nowackiego, Kraków",None,siłownia zewnętrzna,None
SP nr 32 im. Karola Chodkiewicza,SP nr 32 im. Karola Chodkiewicza,None,None,SP nr 32 im. Karola Chodkiewicza,None,boisko wielofunkcyjne,zajęcia sportowe


In [ ]:
del vertical_concat['Facility.Name'] 

In [ ]:
vertical_concat.to_excel('Final_krakawo_facilities.xlsx',index=False)     

In [ ]:
pd.read_csv('Desktop/web_scraping/Final_krakawo_facilities.csv') 

,Facility.Name,Facility.Tel_Number,Facility.Web_site,Facility.Adress,Facility.email,Facility.Courts,Facility.Sport_activities
0,Klub Sportowy Korona,+48 12 656 53 89,NaN,"ul. Kalwaryjska 9, 30-504 Kraków",NaN,"ścianka wspinaczkowa, basen kryty, sala gimnas...","wspinaczka, pływanie, gimnastyka, koszykówka, ..."
1,Ludowy Klub Sportowy Tonianka,+48 604 510 041,"http://tonianka.futbolowo.pl/menu,2,o-klubie.html","ul. Skotnica 1, 30-394 Kraków",NaN,"boisko do piłki nożnej, boisko wielofunkcyjne","piłka nożna, koszykówka"
2,A3 Sport Fitness,+48 12 416 10 19,NaN,"ul. Konecznego 4, 31-216 Kraków",NaN,siłownia wewnętrzna,fitness
3,Active Niuton Ośrodek Promocji Zdrowia,+48 500 210 387,http://activeniuton.pl,"Osiedle Teatralne 10, 31-943 Kraków",NaN,siłownia wewnętrzna,fitness
4,Akademia Górniczo Hutnicza,+48 12 617 48 40,http://www.basen.agh.edu.pl/?page_id=187,"ul. Buszka 4, 30-150 Kraków",NaN,"centrum zabaw, sala sportowa, basen kryty, krę...","plac zabaw dla dzieci, Tai Chi, fitness, pływa..."
...,...,...,...,...,...,...,...
664,Krakowski park linowy,+48 514 256 647,NaN,"ul. Widłakowa 13, 30-380 Kraków",NaN,park linowy,park linowy
665,Park Planty im. F. Nowackiego,NaN,NaN,"Park Planty im. F. Nowackiego, Kraków",NaN,siłownia zewnętrzna,NaN
666,SP nr 32 im. Karola Chodkiewicza,NaN,NaN,SP nr 32 im. Karola Chodkiewicza,NaN,boisko wielofunkcyjne,zajęcia sportowe
667,Ścieżka edukacyjna,+48 12 201 02 40,NaN,"Mistrzejowice, Kraków",NaN,"plac zabaw dla dzieci, ścieżka edukacyjna","plac zabaw dla dzieci, ścieżka edukacyjna"


In [2]:
(2000000*0.06)/5

24000.0